In [1]:
# pip install biopython

In [2]:
# rm -r /content/ZeaMays-TEsClassification

In [3]:
import os
import zipfile
import glob

import pandas as pd
from Bio import SeqIO

In [4]:
# !git clone https://github.com/vini-mon/ZeaMays-TEsClassification.git

In [5]:
# fasta = "ZMays/fasta"
# tes = "ZMays/tes"
# groups = "ZMays/Agrup"

fasta = "ZMays/fasta"
tes = "ZMays/tes"
groups = "ZMays/Agrup"

In [6]:
# Navigate to the directory containing the zip files
os.chdir(fasta)

# Iterate through the zip files
for i in range(1, 11):
  zip_filename = f"chromosome{i}.zip"

  #Check if the zip file exists
  if os.path.exists(zip_filename):
    try:
      with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall()
        print(f"Extração bem-sucedida {zip_filename}")
    except zipfile.BadZipFile:
      print(f"Erro: {zip_filename} não é um arquivo zip.")
    except Exception as e:
      print(f"Ocorreu um erro durante a extração {zip_filename}: {e}")
  else:
    print(f"Warning: {zip_filename} não encontrado.")

# Use glob to find all .zip files
zip_files = glob.glob("*.zip")

# Iterate through the found zip files
for zip_filename in zip_files:
    try:
        os.remove(zip_filename)
        print(f"Removido com sucesso {zip_filename}")
    except OSError as e:
        print(f"Erro: {zip_filename}: {e}")

In [7]:
cd ../..

c:\Users\Vinícius\Documents\USP\ZeaMays-TEsClassification


c:\Users\Vinícius\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
ls

 O volume na unidade C n�o tem nome.
 O N�mero de S�rie do Volume � C6A6-CF4A

 Pasta de c:\Users\Vin�cius\Documents\USP\ZeaMays-TEsClassification

02/06/2025  15:46    <DIR>          .
01/06/2025  05:31    <DIR>          ..
14/05/2025  03:27               136 .gitattributes
11/05/2025  17:07           775.257 2024-Trabalho-Classificacao-TEs.pdf
14/05/2025  16:58               756 analise_metricas.txt
01/06/2025  15:03       284.885.801 df_metricas_seq.csv
02/06/2025  15:23            16.330 k-mer_csv.ipynb
30/05/2025  16:02            23.697 mathfeature_processing_zmays.py
14/05/2025  02:35             3.620 partial_metrics_agrup.py
11/05/2025  17:07               152 README.md
02/06/2025  15:50           422.973 train_RF.ipynb
02/06/2025  15:45           419.784 train_XGBoost.ipynb
02/06/2025  15:55            19.409 ZeaMays_BioInfo2.ipynb
14/05/2025  02:52    <DIR>          ZMays
              11 arquivo(s)    286.567.915 bytes
               3 pasta(s)   12.189.016.064 bytes dispon

In [9]:
file_path = (tes + "/TEAnnotationFinal_Helitron.gff3")

with open(file_path, "r") as file:
    # Use list comprehension para pegar as primeiras 10 linhas
    for i, line in enumerate(file):
        print(line.strip())  # strip() remove espaços extras e quebras de linha duplicadas
        if i == 9:  # Exibe apenas as primeiras 10 linhas
            break

5	APTEdb	Class II subclass 2/Helitron/Helitron	28903351	28903636	.	-	.	TE-Score=0.285;Software=RepeatModeler;Length=285bps
3	APTEdb	Class II subclass 2/Helitron/Helitron	2184634	2185235	.	-	.	TE-Score=0.428;Software=RepeatMasker;Length=601bps
7	APTEdb	Class II subclass 2/Helitron/Helitron	82856122	82857584	.	+	.	TE-Score=0.428;Software=RepeatModeler;Length=1462bps
1	APTEdb	Class II subclass 2/Helitron/Helitron	239302471	239302834	.	+	.	TE-Score=0.285;Software=RepeatModeler;Length=363bps
10	APTEdb	Class II subclass 2/Helitron/Helitron	5646459	5648982	.	-	.	TE-Score=0.571;Software=HelitronScanner;Length=2523bps
1	APTEdb	Class II subclass 2/Helitron/Helitron	185134318	185151509	.	-	.	TE-Score=0.714;Software=HelitronScanner;Length=17191bps
2	APTEdb	Class II subclass 2/Helitron/Helitron	240141757	240157943	.	-	.	TE-Score=0.714;Software=HelitronScanner;Length=16186bps
8	APTEdb	Class II subclass 2/Helitron/Helitron	2261318	2261604	.	+	.	TE-Score=0.142;Software=RepeatModeler;Length=286bps
2	AP

In [10]:
def add_gff3_to_dataframe(df, file_path):
    """
    Lê um arquivo GFF3, extrai os dados e adiciona ao DataFrame existente.

    Parâmetros:
        df (pd.DataFrame): DataFrame existente para o qual os dados serão adicionados.
        file_path (str): Caminho do arquivo GFF3.

    Retorna:
        pd.DataFrame: DataFrame atualizado com os novos dados.
    """
    # Definir os nomes das colunas do arquivo GFF3
    column_names = ["Chr", "SourceAnnotation", "COS", "Start", "End", "Score", "Strand", "Phase", "Attributes"]

    # Lista para armazenar os dados do novo arquivo
    data = []

    try:
        with open(file_path, "r") as file:
            for line in file:
                if not line.startswith("#"):  # Ignorar linhas de comentários
                    parts = line.strip().split("\t")
                    if len(parts) == len(column_names):  # Verificar o número correto de colunas
                        data.append(parts)
                    else:
                        print(f"Warning: Skipping line with incorrect number of columns: {line.strip()}")

        # Criar um DataFrame com os dados lidos
        new_df = pd.DataFrame(data, columns=column_names)

        # Adicionar os novos dados ao DataFrame existente
        updated_df = pd.concat([df, new_df], ignore_index=True)
        return updated_df

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

    return df  # Retorna o DataFrame original caso haja um erro


In [11]:
# helitron = "/content/ZeaMays-TEsClassification/ZMays/tes/TEAnnotationFinal_Helitron.gff3"
# line = "/content/ZeaMays-TEsClassification/ZMays/tes/TEAnnotationFinal_LINE.gff3"
# ltr = "/content/ZeaMays-TEsClassification/ZMays/tes/TEAnnotationFinal_LTR.gff3"
# mite = "/content/ZeaMays-TEsClassification/ZMays/tes/TEAnnotationFinal_MITE.gff3"
# sine = "/content/ZeaMays-TEsClassification/ZMays/tes/TEAnnotationFinal_SINE.gff3"
# tir = "/content/ZeaMays-TEsClassification/ZMays/tes/TEAnnotationFinal_TIR.gff3"

helitron = "ZMays/tes/TEAnnotationFinal_Helitron.gff3"
line = "ZMays/tes/TEAnnotationFinal_LINE.gff3"
ltr = "ZMays/tes/TEAnnotationFinal_LTR.gff3"
mite = "ZMays/tes/TEAnnotationFinal_MITE.gff3"
sine = "ZMays/tes/TEAnnotationFinal_SINE.gff3"
tir = "ZMays/tes/TEAnnotationFinal_TIR.gff3"

In [12]:
df = ""
df = pd.DataFrame(columns=["Chr", "SourceAnnotation", "COS", "Start", "End", "Score", "Strand", "Phase", "Attributes"])

df = add_gff3_to_dataframe(df, helitron)
df = add_gff3_to_dataframe(df, line)
df = add_gff3_to_dataframe(df, ltr)
df = add_gff3_to_dataframe(df, mite)
df = add_gff3_to_dataframe(df, sine)
df = add_gff3_to_dataframe(df, tir)

print(df.head())

  Chr SourceAnnotation                                    COS      Start  \
0   5           APTEdb  Class II subclass 2/Helitron/Helitron   28903351   
1   3           APTEdb  Class II subclass 2/Helitron/Helitron    2184634   
2   7           APTEdb  Class II subclass 2/Helitron/Helitron   82856122   
3   1           APTEdb  Class II subclass 2/Helitron/Helitron  239302471   
4  10           APTEdb  Class II subclass 2/Helitron/Helitron    5646459   

         End Score Strand Phase  \
0   28903636     .      -     .   
1    2185235     .      -     .   
2   82857584     .      +     .   
3  239302834     .      +     .   
4    5648982     .      -     .   

                                          Attributes  
0  TE-Score=0.285;Software=RepeatModeler;Length=2...  
1  TE-Score=0.428;Software=RepeatMasker;Length=60...  
2  TE-Score=0.428;Software=RepeatModeler;Length=1...  
3  TE-Score=0.285;Software=RepeatModeler;Length=3...  
4  TE-Score=0.571;Software=HelitronScanner;Length...  


In [13]:
#df = df[df['Strand'] == '+']
df.reset_index(drop=True, inplace=True)

print(df.head())
print(f"\n\nNúmero de entradas: {len(df)}")

  Chr SourceAnnotation                                    COS      Start  \
0   5           APTEdb  Class II subclass 2/Helitron/Helitron   28903351   
1   3           APTEdb  Class II subclass 2/Helitron/Helitron    2184634   
2   7           APTEdb  Class II subclass 2/Helitron/Helitron   82856122   
3   1           APTEdb  Class II subclass 2/Helitron/Helitron  239302471   
4  10           APTEdb  Class II subclass 2/Helitron/Helitron    5646459   

         End Score Strand Phase  \
0   28903636     .      -     .   
1    2185235     .      -     .   
2   82857584     .      +     .   
3  239302834     .      +     .   
4    5648982     .      -     .   

                                          Attributes  
0  TE-Score=0.285;Software=RepeatModeler;Length=2...  
1  TE-Score=0.428;Software=RepeatMasker;Length=60...  
2  TE-Score=0.428;Software=RepeatModeler;Length=1...  
3  TE-Score=0.285;Software=RepeatModeler;Length=3...  
4  TE-Score=0.571;Software=HelitronScanner;Length...  




In [14]:
# Ordernar o DataFrame
df['Start'] = df['Start'].astype(int)
df['End'] = df['End'].astype(int)
df_sorted = df.sort_values(by=['Chr', 'Start'])

# Reset the index of the sorted DataFrame
df_sorted.reset_index(drop=True, inplace=True)

print(df_sorted.head())
print(f"\n\nNúmero de entradas: {len(df_sorted)}")

  Chr SourceAnnotation                                    COS  Start    End  \
0   1           APTEdb  Class II subclass 2/Helitron/Helitron  27169  31243   
1   1           APTEdb                    Class I/LTR/Unknown  31390  31484   
2   1           APTEdb                      Class I/LTR/Gypsy  31405  31493   
3   1           APTEdb                    Class I/LTR/Unknown  31573  31667   
4   1           APTEdb                      Class I/LTR/Gypsy  31596  31679   

  Score Strand Phase                                         Attributes  
0     .      +     .  TE-Score=0.714;Software=HelitronScanner;Length...  
1     .      -   287  TE-Score=0.285;Software=LTRRetriever;Length=94bps  
2     .      +     .  TE-Score=0.428;Software=RepeatModeler;Length=8...  
3     .      -   281  TE-Score=0.142;Software=LTRRetriever;Length=94bps  
4     .      +     .  TE-Score=0.142;Software=RepeatModeler;Length=8...  


Número de entradas: 678138


In [15]:
# Lista dos tipos permitidos
types = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]

# Filtrar o DataFrame para manter apenas os tipos permitidos
df = df[df["Chr"].isin(types)]
df.reset_index(drop=True, inplace=True)

# Sumarização
summary = df["Chr"].value_counts()

for index, value in summary.items():
    print(f"{index}: {value}")

print(f"\n\nNúmero de entradas: {len(df)}")

4: 100427
1: 97069
2: 74309
3: 72265
5: 68571
8: 55876
7: 55250
6: 51638
9: 49783
10: 46214


Número de entradas: 671402


In [16]:
df.head()

,Chr,SourceAnnotation,COS,Start,End,Score,Strand,Phase,Attributes
0,5,APTEdb,Class II subclass 2/Helitron/Helitron,28903351,28903636,.,-,.,TE-Score=0.285;Software=RepeatModeler;Length=2...
1,3,APTEdb,Class II subclass 2/Helitron/Helitron,2184634,2185235,.,-,.,TE-Score=0.428;Software=RepeatMasker;Length=60...
2,7,APTEdb,Class II subclass 2/Helitron/Helitron,82856122,82857584,.,+,.,TE-Score=0.428;Software=RepeatModeler;Length=1...
3,1,APTEdb,Class II subclass 2/Helitron/Helitron,239302471,239302834,.,+,.,TE-Score=0.285;Software=RepeatModeler;Length=3...
4,10,APTEdb,Class II subclass 2/Helitron/Helitron,5646459,5648982,.,-,.,TE-Score=0.571;Software=HelitronScanner;Length...


In [17]:
dfs_metrics = []

i = 0
for f in glob.glob(groups + "/*.csv"):
  df_metric = pd.read_csv(f)
  df_metric = df_metric.sort_values(by='nameseq')

  columns = [f"{f.replace('ZMays/Agrup/', '').replace('.csv', '')}_{column}" for column in df_metric.columns]
  if i != 0:
    df_metric.drop(columns=['nameseq'], inplace=True)
    columns.remove(columns[0])
  else:
    columns[0] = 'nameseq'

  df_metric.columns = columns

  df_metric.reset_index(drop=True, inplace=True)
  dfs_metrics.append(df_metric)
  i += 1

df_metrics = pd.concat(dfs_metrics, axis=1)
df_metrics.head()

,nameseq,ZMays/Agrup\anf_fourier_average,ZMays/Agrup\anf_fourier_median,ZMays/Agrup\anf_fourier_maximum,ZMays/Agrup\anf_fourier_minimum,ZMays/Agrup\anf_fourier_peak,ZMays/Agrup\anf_fourier_none_levated_peak,ZMays/Agrup\anf_fourier_sample_standard_deviation,ZMays/Agrup\anf_fourier_population_standard_deviation,ZMays/Agrup\anf_fourier_percentile15,...,ZMays/Agrup\orf_maximum_ORF_length,ZMays/Agrup\orf_minimum_ORF_length,ZMays/Agrup\orf_std_ORF_length,ZMays/Agrup\orf_average_ORF_length,ZMays/Agrup\orf_cv_ORF_length,ZMays/Agrup\orf_maximum_GC_content_ORF,ZMays/Agrup\orf_minimum_GC_content_ORF,ZMays/Agrup\orf_std_GC_content_ORF,ZMays/Agrup\orf_average_GC_content_ORF,ZMays/Agrup\orf_cv_GC_content_ORF
0,10_104431_104500,8.248204,0.495401,481.523442,0.001764,2.828899,18.355640,57.003340,57.414922,0.166519,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,10_104817_104874,10.103667,2.135769,390.545718,0.015395,1.913497,10.105725,50.544025,50.985466,0.400310,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,10_104957_105096,13.674118,1.140519,1674.002251,0.014161,3.412774,32.829516,140.838334,141.344039,0.170290,...,15.0,15.0,0.000000,15.000000,0.000000,26.666667,26.666667,0.000000,26.666667,0.000000
3,10_106081_106190,13.922727,1.439795,1292.065720,0.055156,2.633584,22.574048,122.443130,123.003513,0.403268,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,10_106081_108136,171.914850,10.534463,322090.374942,0.038343,3.986470,183.905772,7101.353795,7103.081408,2.586421,...,90.0,9.0,21.175513,32.538462,0.650784,44.444444,16.666667,8.278575,32.995014,0.250904


In [18]:
print(df_metrics['nameseq'].astype(str))

0        10_104431_104500
1        10_104817_104874
2        10_104957_105096
3        10_106081_106190
4        10_106081_108136
               ...       
62536                 nan
62537                 nan
62538                 nan
62539                 nan
62540                 nan
Name: nameseq, Length: 62541, dtype: object


In [19]:
mask = df_metrics['nameseq'].apply(lambda x: isinstance(x, str))
df_filtrado = df_metrics.loc[mask]

In [20]:
mascara = df_metrics['nameseq'].map(type) == str

# 2. Aplica a máscara para filtrar o DataFrame
df_metrics_filtrado = df_metrics[mascara]

print("DataFrame original:")
print(df_metrics)
print("\nDataFrame filtrado (apenas strings em 'nameseq'):")
print(df_metrics_filtrado)

DataFrame original:
                nameseq  ZMays/Agrup\anf_fourier_average  \
0      10_104431_104500                         8.248204   
1      10_104817_104874                        10.103667   
2      10_104957_105096                        13.674118   
3      10_106081_106190                        13.922727   
4      10_106081_108136                       171.914850   
...                 ...                              ...   
62536               NaN                              NaN   
62537               NaN                              NaN   
62538               NaN                              NaN   
62539               NaN                              NaN   
62540               NaN                              NaN   

       ZMays/Agrup\anf_fourier_median  ZMays/Agrup\anf_fourier_maximum  \
0                            0.495401                       481.523442   
1                            2.135769                       390.545718   
2                            1.140519

In [22]:
print(df)

       Chr SourceAnnotation                                    COS      Start  \
0        5           APTEdb  Class II subclass 2/Helitron/Helitron   28903351   
1        3           APTEdb  Class II subclass 2/Helitron/Helitron    2184634   
2        7           APTEdb  Class II subclass 2/Helitron/Helitron   82856122   
3        1           APTEdb  Class II subclass 2/Helitron/Helitron  239302471   
4       10           APTEdb  Class II subclass 2/Helitron/Helitron    5646459   
...     ..              ...                                    ...        ...   
671397   1           APTEdb         Class II subclass 1/TIR/Ginger  223982226   
671398   4           APTEdb         Class II subclass 1/TIR/Ginger  118840026   
671399   4           APTEdb         Class II subclass 1/TIR/Ginger    7068255   
671400   4           APTEdb         Class II subclass 1/TIR/Ginger   13348879   
671401   1           APTEdb         Class II subclass 1/TIR/Ginger  253818258   

              End Score Str

In [23]:
print(df_metrics_filtrado)

                   nameseq  ZMays/Agrup\anf_fourier_average  \
0         10_104431_104500                         8.248204   
1         10_104817_104874                        10.103667   
2         10_104957_105096                        13.674118   
3         10_106081_106190                        13.922727   
4         10_106081_108136                       171.914850   
...                    ...                              ...   
62475  9_80083385_80083492                         9.731974   
62476  9_80085082_80085224                        12.432436   
62477  9_80090125_80096282                       418.358525   
62478  9_80104841_80126544                      1434.734350   
62479    9_8012224_8012430                        17.099596   

       ZMays/Agrup\anf_fourier_median  ZMays/Agrup\anf_fourier_maximum  \
0                            0.495401                     4.815234e+02   
1                            2.135769                     3.905457e+02   
2                    

In [28]:
def juntar_csvs_em_dataframe(arquivo_csv1: str, arquivo_csv2: str) -> pd.DataFrame | None:

    try:
        print(f"\nLendo o primeiro arquivo para junção: '{arquivo_csv1}'...")
        df1 = pd.read_csv(arquivo_csv1)
        print(f"Dimensões de '{os.path.basename(arquivo_csv1)}': {df1.shape[0]} linhas, {df1.shape[1]} colunas")

        print(f"Lendo o segundo arquivo para junção: '{arquivo_csv2}'...")
        df2 = pd.read_csv(arquivo_csv2)
        print(f"Dimensões de '{os.path.basename(arquivo_csv2)}': {df2.shape[0]} linhas, {df2.shape[1]} colunas")

        df_concatenado = pd.concat([df1, df2], ignore_index=True)
        print(f"Dimensões do DataFrame final concatenado: {df_concatenado.shape[0]} linhas, {df_concatenado.shape[1]} colunas")
        print(f"Arquivos '{arquivo_csv1}' e '{arquivo_csv2}' juntados em um DataFrame com sucesso!")
        return df_concatenado
    except FileNotFoundError:
        print(f"Erro: Um ou ambos os arquivos CSV ('{arquivo_csv1}', '{arquivo_csv2}') não foram encontrados.")
        return None
    except Exception as e:
        print(f"Ocorreu um erro ao juntar os arquivos CSV: {e}")
        return None
    
def juntar_dataframes_inner(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame | None:
    
    if 'nameseq' not in df1.columns or 'nameseq' not in df2.columns:
        print("Erro: A coluna 'nameseq' não está presente em ambos os DataFrames.")
        return None
    
    df_juntado = pd.merge(df1, df2, on='nameseq', how='inner')
    print(f"DataFrames juntados com 'inner join' na coluna 'nameseq'. Dimensões do resultado: {df_juntado.shape}")
    return df_juntado

In [29]:
arquivo_csv_parte1 = 'ZMays/Agrup/k-mer/parte1.csv'
arquivo_csv_parte2 = 'ZMays/Agrup/k-mer/parte2.csv'

df_kmer = juntar_csvs_em_dataframe(arquivo_csv_parte1, arquivo_csv_parte2)

df_all_metrics = juntar_dataframes_inner(df_metrics_filtrado, df_kmer)

print(df_all_metrics.head())


Lendo o primeiro arquivo para junção: 'ZMays/Agrup/k-mer/parte1.csv'...
Dimensões de 'parte1.csv': 31241 linhas, 5462 colunas
Lendo o segundo arquivo para junção: 'ZMays/Agrup/k-mer/parte2.csv'...
Dimensões de 'parte2.csv': 31241 linhas, 5462 colunas
Dimensões do DataFrame final concatenado: 62482 linhas, 5462 colunas
Arquivos 'ZMays/Agrup/k-mer/parte1.csv' e 'ZMays/Agrup/k-mer/parte2.csv' juntados em um DataFrame com sucesso!
DataFrames juntados com 'inner join' na coluna 'nameseq'. Dimensões do resultado: (62480, 5758)
            nameseq  ZMays/Agrup\anf_fourier_average  \
0  10_104431_104500                         8.248204   
1  10_104817_104874                        10.103667   
2  10_104957_105096                        13.674118   
3  10_106081_106190                        13.922727   
4  10_106081_108136                       171.914850   

   ZMays/Agrup\anf_fourier_median  ZMays/Agrup\anf_fourier_maximum  \
0                        0.495401                       481.52344

In [30]:
print(df_all_metrics.shape)

(62480, 5758)


In [34]:
import pandas as pd
import numpy as np # Para pd.NA ou np.nan, se necessário

def join_metrics_inner(df_main, df_metrics_values):
    
    df_metrics = df_metrics_values.copy()

    # Dividir 'nameseq' em colunas chave
    try:
        
        split_data = df_metrics['nameseq'].str.split('_', expand=True, n=2)
        
        if split_data.shape[1] < 3:
            raise ValueError(
                "A coluna 'nameseq' em df_metrics_values não pôde ser dividida em 3 partes (Chr, Start, End) consistentemente."
            )
        
        df_metrics['_Chr_key_metric'] = split_data[0].astype(str)
        df_metrics['_Start_key_metric'] = split_data[1].astype(str)
        df_metrics['_End_key_metric'] = split_data[2].astype(str)
        
    except Exception as e:
        
        print(f"Erro ao processar a coluna 'nameseq': {e}")
        print("Verifique o formato dos dados em 'nameseq'. Esperado: 'Chr_Start_End'.")
       
        raise

    # Identificar as colunas de métricas que serão transferidas/atualizadas
    metric_value_cols = [
        col for col in df_metrics.columns
        if col not in ['nameseq', '_Chr_key_metric', '_Start_key_metric', '_End_key_metric']
    ]
    
    # Selecionar apenas as colunas chave e de métricas para a junção
    df_metrics_to_join = df_metrics[
        ['_Chr_key_metric', '_Start_key_metric', '_End_key_metric'] + metric_value_cols
    ]

    df_left = df_main.copy()
    
    # Adicionar colunas chave temporárias em df_left, convertendo para string para correspondência
    df_left['_Chr_key_df'] = df_left['Chr'].astype(str)
    df_left['_Start_key_df'] = df_left['Start'].astype(str)
    df_left['_End_key_df'] = df_left['End'].astype(str)

    merged_df = pd.merge(
        df_left,
        df_metrics_to_join,
        left_on=['_Chr_key_df', '_Start_key_df', '_End_key_df'],
        right_on=['_Chr_key_metric', '_Start_key_metric', '_End_key_metric'],
        how='inner', # Especifica a junção interna
        suffixes=('', '_newmetric')
    )

    
    for metric_col_name in metric_value_cols:
        
        suffixed_metric_col_name = metric_col_name + '_newmetric'
        
        if suffixed_metric_col_name in merged_df.columns:
            
            merged_df[metric_col_name] = merged_df[suffixed_metric_col_name]
            merged_df.drop(columns=[suffixed_metric_col_name], inplace=True)
            
    cols_to_drop = [
        '_Chr_key_df', '_Start_key_df', '_End_key_df',
        '_Chr_key_metric', '_Start_key_metric', '_End_key_metric' # Remover se ainda existirem
    ]
    
    existing_cols_to_drop = [col for col in cols_to_drop if col in merged_df.columns]
    merged_df.drop(columns=existing_cols_to_drop, inplace=True, errors='ignore')
    
    return merged_df

In [35]:
df_resultado_juncao = join_metrics_inner(df, df_all_metrics)
print(df_resultado_juncao)

      Chr SourceAnnotation                                    COS      Start  \
0       1           APTEdb  Class II subclass 2/Helitron/Helitron  185134318   
1       1           APTEdb  Class II subclass 2/Helitron/Helitron  108502298   
2       9           APTEdb  Class II subclass 2/Helitron/Helitron   77998293   
3       4           APTEdb  Class II subclass 2/Helitron/Helitron  206727217   
4       8           APTEdb  Class II subclass 2/Helitron/Helitron  165674147   
...    ..              ...                                    ...        ...   
67681   4           APTEdb    Class II subclass 1/TIR/hAT-Charlie  233317063   
67682   6           APTEdb    Class II subclass 1/TIR/hAT-Charlie  164032515   
67683   4           APTEdb    Class II subclass 1/TIR/hAT-Charlie  200057025   
67684   2           APTEdb    Class II subclass 1/TIR/hAT-Charlie   13648304   
67685   4           APTEdb    Class II subclass 1/TIR/hAT-Charlie  233261821   

             End Score Strand Phase  \


In [ ]:
df_met_seq = 'df_metricas_seq_final.csv'

try:
    df_resultado_juncao.to_csv(df_met_seq, index=False, encoding='utf-8')
    print(f"DataFrame salvo com sucesso como '{df_met_seq}'!")
except Exception as e:
    print(f"Ocorreu um erro ao salvar o DataFrame: {e}")

DataFrame salvo com sucesso como 'df_metricas_seq.csv'!
